# Twitter Search
- Search over all user Tweets.

## Capacity

Assume
- 1.5B total users, 800M daily active users.
- 400M tweets per day.
- Each Tweet is 300 bytes.
- 500M searches per day.
- Search query contains multiple words combined with AND/OR.

Storage
- (400M tweets/day) * (300 bytes/tweet) = 120GB/day = 1.38MB/s
- In 5 years, 120GB * 365 days * 5 years = 200TB
- Apply safety factor, so aim at 250TB
- If extra copy for fault tolerance, 500TB

## API
- search(api_dev_key, search_terms, max_results_to_return)
    - Returns JSON with list of Tweets matching the search query.

## Design
- Two columns: TweetId and TweetText.
- Need 400M * 365 days * 5 years => 730 billion TweetIds (5 bytes)
- Feed TweetId into a hash function to identify a server to store Tweet.

Index
- Distributed hash table where key is word and value is list of TweetIds whose Tweet contains that word.
- We already did "Feed TweetId into a hash function to identify a server to store Tweet".
- Then, for each server that stores Tweets, build an index on that server.
- When serving queries, aggregate result from each server and return them to users.

<img src="img/twitter_search1.png" style="width:600px;height:800px;">

Fault Tolerance
- Maintain replicas for each index server.
- If primay and backups die all together, rebuild index server from a reverse hash table.
    - Key is index server number and value is all TweetIds (In a form of hash)
    
## Caching
- Use LRU eviction policy.